### **1. 필요 라이브러리 다운 및 임포트**

In [ ]:
!pip install -U pymilvus
!pip install --upgrade openai
!pip install -U sentence-transformers

In [ ]:
from pymilvus import MilvusClient
from pymilvus import FieldSchema, DataType
from pymilvus import FieldSchema, CollectionSchema

import pandas as pd
import numpy as np
import time
import openai
from openai import OpenAI
import os

### **2. DB 생성 및 컬렉션 생성**

In [72]:
INDEX_TYPE = "FLAT"
DIMENSION = 1024
METRIC_TYPE = "IP"
NUM_PARTITIONS = 17

class MakeCollections:
  def __init__(self, client, index_type, metric_type, dimension):
    self.client = client
    self.index_type = index_type
    self.metric_type = metric_type
    self.dimension = dimension


  # 스키마 생성
  def create_schema(self):
    fields = [
      FieldSchema(name="id", dtype=DataType.INT64, is_primary=True),
      FieldSchema(name="place_name", dtype=DataType.VARCHAR, max_length=100, description="the name of place"),
      FieldSchema(name="category", dtype=DataType.VARCHAR, max_length=30, description="the category of place"),
      FieldSchema(name="area_name", dtype=DataType.VARCHAR, max_length=100, description="the name of administrative district"),
      FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=2000, description="elements of travel sites"),
      FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=self.dimension, description="vector"),
    ]
    schema = CollectionSchema(fields=fields, description="travel sites", partition_key_field="area_name")
    return schema

  # 인덱스 생성
  def create_index(self):
    index_params = self.client.prepare_index_params()

    index_params.add_index(
      field_name="embedding",
      index_type=self.index_type,
      metric_type=self.metric_type
    )

    return index_params

  # 컬렉션 생성
  def create_collection(self, collection_name):
    self.client.create_collection(
      collection_name=collection_name,
      schema=self.create_schema(),
      index_params=self.create_index(),
      num_partitions=NUM_PARTITIONS
    )

    time.sleep(2)

    res = self.client.get_load_state(
      collection_name=collection_name
    )
    print(res)

    return self.client


In [73]:
from google.colab import userdata
url = userdata.get("URL")

# 데이터베이스 연결
client = MilvusClient(url)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 3d536c3ce12e44fc83e63952c200895c


In [74]:
collection = MakeCollections(client, INDEX_TYPE, METRIC_TYPE, DIMENSION)

if client.has_collection(collection_name="kstartup_travel_sites") or client.has_collection(collection_name="nowlocal_travel_sites"):
    client.drop_collection(
        collection_name="kstartup_travel_sites"
    )

    client.drop_collection(
        collection_name="nowlocal_travel_sites"
    )

kstartup_collection = collection.create_collection("kstartup_travel_sites")
nowlocal_collection = collection.create_collection("nowlocal_travel_sites")

DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: kstartup_travel_sites
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: kstartup_travel_sites
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: nowlocal_travel_sites


{'state': <LoadState: Loaded>}


DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: nowlocal_travel_sites


{'state': <LoadState: Loaded>}


In [75]:
# 만들어진 컬렉션 확인
client.list_collections()

['kstartup_travel_sites', 'nowlocal_travel_sites']

In [76]:
# 만든 컬렉션 확인
res = client.describe_collection(
    collection_name="kstartup_travel_sites"
)
res

{'collection_name': 'kstartup_travel_sites',
 'auto_id': False,
 'num_shards': 1,
 'description': 'travel sites',
 'fields': [{'field_id': 100,
   'name': 'id',
   'description': '',
   'type': <DataType.INT64: 5>,
   'params': {},
   'is_primary': True},
  {'field_id': 101,
   'name': 'place_name',
   'description': 'the name of place',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 100}},
  {'field_id': 102,
   'name': 'category',
   'description': 'the category of place',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 30}},
  {'field_id': 103,
   'name': 'area_name',
   'description': 'the name of administrative district',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 100},
   'is_partition_key': True},
  {'field_id': 104,
   'name': 'text',
   'description': 'elements of travel sites',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 2000}},
  {'field_id': 105,
   'name': 'embedding',
   'description': 'vector',
   'typ

### **3. 데이터 셋 생성 및 입력**


In [77]:
# 임베딩 모델 로드
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("upskyy/bge-m3-korean")

#### **3-1. 강원도 지역이 아닌 다른 지역(서울특별시, 부산 광역시 등)의 장소 데이터 생성 및 입력**
이유: 강원도 지역만 추가 데이터가 존재하기 때문에 컬렉션에 저장되는 문서 내용이 다름.
따라서, 강원도 지역 외의 장소는 다른 코드를 이용해 데이터 셋을 생성

In [78]:
# 데이터 가져올 파일명
data_list = {
    # 컬렉션 이름 : 가져올 데이터 파일이름
    'kstartup_travel_sites' : 'Kstartup_data_fin.xlsx',
    'nowlocal_travel_sites' : 'Nowlocal_data_fin.xlsx'
}


# 행정구역 구분
sections_1 = ["서울특별시", "부산광역시", "인천광역시", "대구광역시", "대전광역시",
            "광주광역시", "울산광역시", "세종특별자치시", "경기도", "충청북도",
            "충청남도", "전라남도", "경상북도", "경상남도", "전북특별자치도", "제주특별자치도"]


def make_dataset_1(row):
    id          = row['store ID']
    place_name  = row['상호명']
    category    = row['카테고리']
    area_name   = row['주소1'].split()[0]

    document = f"""- 장소명: {row['상호명']}
  카테고리: {row['카테고리']}
  장소 키워드: {row['키워드']}
  위치: {row['주소2']}
  해시태그: {row['해시태그']}"""

    vector = model.encode(document)
    return {'id': id, 'place_name':place_name, 'category':category, 'area_name':area_name, 'text':document, 'embedding':vector}

def make_document_hashtag_1(row):
  area_1 = row['주소1'].split()[1]
  category = row['카테고리']

  return f"#{area_1} #{category} #추천 여행지 #놀만한 곳 #가볼 만한 곳"


for collection_name, file_name in data_list.items():
  # 엑셀 파일 읽어오기
  original_df = pd.read_excel(file_name, engine='openpyxl')

  # 필요없는 열 버리기
  columns_to_drop = ['설명', '운영정보', '화장실 청결도']
  original_df = original_df.drop(columns=columns_to_drop)

  # 원본 파일 복사, nan값 처리
  copy_df = original_df.copy()
  copy_df = copy_df.replace(np.nan, '')

  for section in sections_1:
    section_df = copy_df[copy_df['주소1'].str.contains(section)].copy()

    # 문서 추가 키워드 및 장소 특징 생성
    section_df["해시태그"] = section_df.apply(make_document_hashtag_1, axis=1)

    # 데이터셋 생성
    dataset = section_df.apply(make_dataset_1, axis=1).tolist()

    '''
    for data in dataset:
      print(data)
    '''

    client.insert(
      collection_name=collection_name,
      data=dataset
    )


#### **3-2. 강원도 지역의 장소 데이터셋 생성 및 입력**

In [79]:
# 데이터 가져올 파일명
data_list = {
    # 컬렉션 이름 : 가져올 데이터 파일이름
    'kstartup_travel_sites' : 'Kstartup_data_fin.xlsx',
    'nowlocal_travel_sites' : 'Nowlocal_data_fin.xlsx'
}


# 행정구역 구분
sections_2 = ["강원특별자치도"]

def make_dataset_2(row):
    id          = row['store ID']
    place_name  = row['상호명']
    category    = row['카테고리']
    area_name   = row['주소1'].split()[0]

    document = f"""- 장소명: {row['상호명']}
  카테고리: {row['카테고리']}
  장소 설명:{row['키워드']}
  장소 분위기: {row['분위기']}
  위치: {row['주소2']}
  서비스: {row['서비스']}
  근처 정류장과의 거리: {row['근처 정류장과의 도보 거리']}
  장소 특징: {row['특징']}
  1인 평균 소비 금액: {row['1인 객단가']}
  해시태그: {row['해시태그']}"""

    vector = model.encode(document)

    return {'id': id, 'place_name':place_name, 'category':category, 'area_name':area_name, 'text':document, 'embedding':vector}


def make_document_hashtag_2(row):
  area_1 = row['주소1'].split()[1]
  place_hashtag = ""

  if row['해시태그'] != '':
    place_hashtag = row['해시태그'].replace(', ',' #')

  return f"#{area_1} {place_hashtag} #추천 여행지 #가볼 만한 곳"


def make_features(row):
    words = [row['반려동물 입장 가능 여부'], row['키즈존 여부'], row['와이파이 여부'], row['휠체어 이용 가능 여부'], row['주차 가능 여부'], row['예약 가능 여부']]
    data = []

    for word in words:
      if word != '':
        data.append(word)

    features = ", ".join(data)
    return features


for collection_name, file_name in data_list.items():
  # 엑셀 파일 읽어오기
  original_df = pd.read_excel(file_name, engine='openpyxl')

  # 필요없는 열 버리기
  columns_to_drop = ['설명', '운영정보', '화장실 청결도']
  original_df = original_df.drop(columns=columns_to_drop)

  # 원본 파일 복사, nan값 처리
  copy_df = original_df.copy()
  copy_df = copy_df.replace(np.nan, '')

  for section in sections_2:
    section_df = copy_df[copy_df['주소1'].str.contains(section)].copy()

    # 문서 추가 키워드 및 장소 특징 생성
    section_df["해시태그"] = section_df.apply(make_document_hashtag_2, axis=1)
    section_df["특징"] = section_df.apply(make_features, axis=1)

    # 필요없는 열 버리기
    columns_to_drop = ['반려동물 입장 가능 여부', '키즈존 여부', '와이파이 여부', '휠체어 이용 가능 여부', '주차 가능 여부', '예약 가능 여부']
    original_df = original_df.drop(columns=columns_to_drop)

    # 데이터셋 생성
    dataset = section_df.apply(make_dataset_2, axis=1).tolist()

    '''
    for data in dataset:
      print(data)
    '''

    client.insert(
      collection_name=collection_name,
      data=dataset
    )


### **4. 삽입된 데이터 결과 확인**




In [80]:
collection_list = ['kstartup_travel_sites', 'nowlocal_travel_sites']

# 컬렉션 구체적 정보(인덱스 지정 필드, 삽입 데이터 개수) 확인
for collection_name in collection_list:
  print(f'-- {collection_name} --')
  res = client.list_indexes(collection_name=collection_name)
  print(f'인덱스 지정된 필드 리스트: {res}')

  res = client.query(
      collection_name=collection_name,
      filter="",
      output_fields=["count(*)"]
  )
  print(f'삽입 데이터 개수: {res}')
  print()


-- kstartup_travel_sites --
인덱스 지정된 필드 리스트: ['embedding']
삽입 데이터 개수: data: ["{'count(*)': 239}"] 

-- nowlocal_travel_sites --
인덱스 지정된 필드 리스트: ['embedding']
삽입 데이터 개수: data: ["{'count(*)': 154}"] 



In [81]:
# 인덱스 정보 확인
for collection_name in collection_list:
  res = client.describe_index(
    collection_name=collection_name,
    index_name="embedding"
  )
  print(res)

  '''
  # 6. Drop index
  client.drop_index(
    collection_name=collection_name,
    index_name="embedding"
  )
  '''


{'index_type': 'FLAT', 'metric_type': 'IP', 'field_name': 'embedding', 'index_name': 'embedding', 'total_rows': 0, 'indexed_rows': 0, 'pending_index_rows': 0, 'state': 'Finished'}
{'index_type': 'FLAT', 'metric_type': 'IP', 'field_name': 'embedding', 'index_name': 'embedding', 'total_rows': 0, 'indexed_rows': 0, 'pending_index_rows': 0, 'state': 'Finished'}


In [82]:
# 필터링 검색 테스트
for collection_name in collection_list:
  res = client.query(
    collection_name=collection_name,
    filter="area_name == '서울특별시'",
    output_fields=["id", "place_name", "text"],
    limit=10
  )
  for data in res:
    print(data)


{'id': 156, 'place_name': '유어플레이스', 'text': '- 장소명: 유어플레이스\n  카테고리: 기타\n  장소 키워드:  리마인드 아현, 웨딩거리로 문화공간 창출\n  위치: 서울 동작구 남부순환로 2069 3층\n  해시태그: #동작구 #기타 #추천 여행지 #놀만한 곳 #가볼 만한 곳'}
{'id': 212, 'place_name': '보틀팩토리', 'text': '- 장소명: 보틀팩토리\n  카테고리: 카페/디저트\n  장소 키워드:  "연희동 로컬과 함꼐하는 플라스틱 제로 마켓 운영"\n  위치: 서울 서대문구 홍연길 26 지하 1층\n  해시태그: #서대문구 #카페/디저트 #추천 여행지 #놀만한 곳 #가볼 만한 곳'}
{'id': 225, 'place_name': '에이디비오', 'text': '- 장소명: 에이디비오\n  카테고리: 기타\n  장소 키워드:  "지역 양조장과 소비자를 연결하는 전통주 큐레이션 미디어 커머스 플랫폼, 주감酒監"\n  위치: 서울 종로구 자하문로 266\n  해시태그: #종로구 #기타 #추천 여행지 #놀만한 곳 #가볼 만한 곳'}
{'id': 229, 'place_name': '올블랑', 'text': '- 장소명: 올블랑\n  카테고리: 기타\n  장소 키워드:  "북촌 한옥마을, 경복궁, 잠실 올림픽공원 등 지역 관광·문화 명소를 배경으로 홈트레이닝 운동 콘텐츠와 관광지를 소개하는 관광콘텐츠를 결합한 미디어 커머스"\n  위치: 서울 중구 청계천로 40 CKL기타지원센터\n  해시태그: #중구 #기타 #추천 여행지 #놀만한 곳 #가볼 만한 곳'}
{'id': 240, 'place_name': '조앤강 ', 'text': '- 장소명: 조앤강 \n  카테고리: 기타\n  장소 키워드:  "도시농업을 이용한 반려동물 식품"\n  위치: 서울 송파구 송파대로 167\n  해시태그: #송파구 #기타 #추천 여행지 #놀만한 곳 #가볼 만한 곳'}
{'id': 242, 'place_name': ' 렉터스

###**5. DB 연결 끊기**

In [83]:
client.close()